<a href="https://colab.research.google.com/github/yw7vvAW611/CS269GroupProject/blob/main/FinBERT_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install textattack[tensorflow] > /dev/null

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
!pip install tqdm > /dev/null

In [ ]:
# Example of sentiment-classification dataset
import pandas
import numpy as np


fpb100_df = pandas.read_csv("semantic_attack_examples.csv")

fpb100_array = np.array(fpb100_df)

fpb100_list = []
for idx in range(len(fpb100_array)):
  true_label = 2
  if "positive" == fpb100_array[idx, 3]:
    true_label = 0
  elif "negative" == fpb100_array[idx, 3]:
    true_label = 1
  else:
    true_label = 2
  cleaned_dp = fpb100_array[idx, 2].replace("[", "").replace("]", "")
  fpb100_list.append((cleaned_dp, true_label))

print(len(fpb100_list))

316


## 1. Improved Genetic Algorithm
 Use improved genetic algorithm augments text by replacing words in original sentence with synonyms from the Glove embedding.

In [9]:
from textattack.augmentation.augmenter import Augmenter
from textattack.constraints.pre_transformation.stopword_modification import StopwordModification
from textattack.shared.word_embeddings import WordEmbedding
from textattack.augmentation import WordNetAugmenter
from textattack.transformations import WordSwapEmbedding
from textattack.constraints.overlap.max_words_perturbed import MaxWordsPerturbed
from textattack.constraints.semantics.word_embedding_distance import WordEmbeddingDistance
import csv
from tqdm import tqdm


#   (search_method): ImprovedGeneticAlgorithm(
#     (pop_size):  60
#     (max_iters):  20
#     (temp):  0.3
#     (give_up_if_no_improvement):  False
#     (post_crossover_check):  False
#     (max_crossover_retries):  20
#     (max_replace_times_per_index):  5
#   )
#   (goal_function):  UntargetedClassification
#   (transformation):  WordSwapEmbedding(
#     (max_candidates):  50
#     (embedding):  WordEmbedding
#   )
#   (constraints): 
#     (0): MaxWordsPerturbed(
#         (max_percent):  0.2
#         (compare_against_original):  True
#       )
#     (1): WordEmbeddingDistance(
#         (embedding):  WordEmbedding
#         (max_mse_dist):  0.5
#         (cased):  False
#         (include_unknown_words):  True
#         (compare_against_original):  False
#       )
#     (2): StopwordModification
#   (is_black_box):  True
# ) 


transformation = WordSwapEmbedding(max_candidates=50)
constraints = [MaxWordsPerturbed(max_percent=0.2, compare_against_original=True),
               WordEmbeddingDistance(max_mse_dist=0.5, cased=False, include_unknown_words=True, compare_against_original=False),
               StopwordModification()]
               
augmenter = Augmenter(transformation=transformation, constraints=constraints, pct_words_to_swap=0.4, transformations_per_example=5)
# create dataframe
f = csv.writer(open("improved_genetic_algo_augmented.csv", "w"))
# Write CSV Header
f.writerow(["id", "sentence", "sentiment"])
idx = 0

for s, label in tqdm(fpb100_list):
  results = augmenter.augment(s)
  f.writerow([idx, s, label])
  idx += 1
  for augmented_s in results:
    f.writerow([idx, augmented_s, label])
    idx += 1
  # print(f"Average Augment USE Score: {results[2]['avg_attack_use_score']}")
  print(f"Augmentations:")
  print(results)

  0%|          | 1/316 [00:01<10:23,  1.98s/it]

Augmentations:
['Cardona slowed her motorcar , turned about and repatriate to the intersection , where she called 911', 'Cardona slowed her motorcar , turned roughly and returning to the intersection , where she phoned 911', 'Cardona slowed her motorcar , turned roughly and reverting to the crossroad , where she telephoned 911', 'Cardona slowed her vehicle , transformed nearly and reverse to the intersection , where she called 911', 'Cardona slowed her vehicle , turned about and reversal to the intersection , where she termed 911']


  1%|          | 2/316 [00:08<16:53,  3.23s/it]

Augmentations:
['Both activity incomes and wisp salts for the three-month calendar growth , respectively from EUR16 .0 m and EUR139m , as compares to the corresponding quarter in 2006', 'Both operating takings and net sales for the three-month timing increased , hetero from EUR16 .0 m and EUR139m , as comparative to the corresponding quarter in 2006', 'Both operations earnings and wisp sales for the three-month periods amplify , separately from EUR16 .0 m and EUR139m , as compared to the corresponding quarter in 2006', 'Both operations income and net sold for the three-month period widens , respectively from EUR16 .0 m and EUR139m , as compare to the corresponding quarter in 2006', 'If operation takings and net sales for the three-month calendar ascent , respectively from EUR16 .0 m and EUR139m , as compares to the corresponding quarter in 2006']


  1%|          | 3/316 [00:13<19:46,  3.79s/it]

...

Augmentations:
["Finn Kemira Cluster 's CEO , Lasse Kurkilahti , says the Groups 's structural reorganisation will unceasingly for at weakest a year", "Finnish Kemira Cluster 's CEO , Lasse Kurkilahti , affirms the Group 's systematic reorganisation will remained for at weakest a year", "Finnish Kemira Clustered 's CEO , Lasse Kurkilahti , say the Grupo 's structural reorganisation will uninterrupted for at weakest a year", "Finnish Kemira Group 's CEO , Lasse Kurkilahti , say the Group 's structure reorganisation will persistent for at lesser a year", "Finnish Kemira Grouping 's CEO , Lasse Kurkilahti , says the Panels 's structural reorganisation will continuing for at less a year"]


100%|█████████▉| 315/316 [32:13<00:07,  7.84s/it]

Augmentations:
["Finland prints circuit boards ( PCBs ) industrial Aspocomp Grupo Oyj confirm on December 4 , 2006 it named Enrique Gilchrist top sex chair of the group 's Asian operational , as of Janeiro 8 , 2007", "Finn printed circuit boards ( PCBs ) industrialists Aspocomp Grupo Oyj affirms on Jul 4 , 2006 it named Enrique Gilchrist upper immodest president of the groups 's Asian operative , as of Janeiro 8 , 2007", "Finnish printed racetrack boards ( PCBs ) manufacturer Aspocomp Grupo Oyj affirms on Juni 4 , 2006 it named Enrique Gilchrist top lewd presidential of the groupings 's Asiatic operations , as of Janeiro 8 , 2007", "Finnish printout racetrack boards ( PCBs ) maker Aspocomp Grupo Oyj highlights on February 4 , 2006 it named Henryk Gilchrist top immodest chairs of the group 's Asian operations , as of Janeiro 8 , 2007", "Finns printed circuit boards ( PCBs ) producers Aspocomp Group Oyj affirms on December 4 , 2006 it named Enrique Gilchrist supremo obscene presidente of

100%|██████████| 316/316 [32:22<00:00,  6.15s/it]

Augmentations:
['Amendments in the market situation and tougher costing competition have radically mitigation demand for bread packaging manufactures at the Kauhava plant , according to the company', 'Change in the market situation and tougher price competitiveness have impressively minimise demand for bread packaging manufactured at the Kauhava plant , according to the company', 'Changes in the market situation and tougher price competitiveness have immeasurably slumps requested for bread packaging manufactured at the Kauhava plant , according to the company', 'Changes in the traded situation and tougher price competitive have freakishly downsizing demand for bread bundle manufactured at the Kauhava plant , according to the company', 'Fourths in the market situation and tougher price competition have substantially diminishing demand for bread bundles manufactured at the Kauhava vegetation , complying to the company']
